In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu



In [ ]:
df_base=pd.read_excel('dataset_780.xlsx',sheet_name=0)
# print(df.columns)

col_1=df_base.columns
# for col in df.columns:
#     print(f'{col} : {df[col].isna().sum()}')

col=['Age','BMI','Sex','Height','Weight','Migratory_Pain','Coughing_Pain','Nausea','Loss_of_Appetite','Dysuria','Stool','Peritonitis','Psoas_Sign','Lower_Right_Abd_Pain','Ipsilateral_Rebound_Tenderness','Contralateral_Rebound_Tenderness','Body_Temperature',
     'WBC_Count','Neutrophil_Percentage','Neutrophilia','CRP','RBC_Count','Thrombocyte_Count','Ketones_in_Urine','Hemoglobin','RDW','Appendix_Diameter','Free_Fluids','Alvarado_Score','Paedriatic_Appendicitis_Score',
     'RBC_in_Urine','WBC_in_Urine','Diagnosis','Severity']




# set_1=set(tuple(all_col))
# set_2=set(tuple(col))
# set_3 = set_1-set_2
# print(set_3)
# print(len(df_base.columns))
# print(len(col))

In [ ]:
df=df_base[col]

df=df.dropna(subset='Diagnosis')


mapping={
    'no':0,
    'yes':1,
    'male':1,
    'female':0,
    'normal':0,
    'diarrhea':1,
    'constipation':1,
    '+':1,
    '++':2,
    '+++':3,
    'local':1,
    'generalized':2,
    'appendicitis':1,
    'no appendicitis':0,
    'uncomplicated':0,
    'complicated':1,
}

for col in df.columns:
    df[col]=df[col].replace(mapping)

df=df[df['Diagnosis']==1]

print (np.mean(df['Age']))

feat_cat=['Sex','Migratory_Pain','Coughing_Pain','Nausea','Loss_of_Appetite','Dysuria','Stool','Neutrophilia',
         'Lower_Right_Abd_Pain','Ipsilateral_Rebound_Tenderness','Contralateral_Rebound_Tenderness',
         'Free_Fluids','Psoas_Sign','Peritonitis','Ketones_in_Urine', 'RBC_in_Urine', 'WBC_in_Urine','Diagnosis', 'Severity']

feat_cont=['Age', 'BMI','Height', 'Weight','Body_Temperature', 'WBC_Count','Neutrophil_Percentage', 
           'RBC_Count','Thrombocyte_Count', 'Hemoglobin', 'RDW','CRP','Appendix_Diameter','Alvarado_Score','Paedriatic_Appendicitis_Score','Diagnosis', 'Severity']

# print (df.shape)
# print(df['Severity'].value_counts())

# print (df['Alvarado_Score'].isna().sum())
# print(df['Severity'].isna().sum())
# print(df['Paedriatic_Appendicitis_Score'].isna().sum())

df_tools=df[['Alvarado_Score', 'Paedriatic_Appendicitis_Score', 'Severity']]
df_tools=df_tools.dropna()
alvarado = df_tools['Alvarado_Score']
pas = df_tools['Paedriatic_Appendicitis_Score']
true = df_tools['Severity']  


auc_as = roc_auc_score(true, alvarado)
auc_pas = roc_auc_score(true, pas)

print(f'AUCROC of Alvarado Score: {auc_as:.2f}')
print(f'AUCROC of Paediatric Appendicitis Score (PAS): {auc_pas:.2f}')


11.082785911008024
AUCROC of Alvarado Score: 0.67
AUCROC of Paediatric Appendicitis Score (PAS): 0.64


C:\Users\ajung\AppData\Local\Temp\ipykernel_13816\2936274877.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col]=df[col].replace(mapping)


In [37]:
#Chi-Square Test for categorical variables

df_cat=df[feat_cat]

target=['Diagnosis', 'Severity']
results_cat=[]
for col in df_cat.columns:
    if col in target:
        continue
    data = df_cat[[col, 'Severity']].dropna()
    x=data.shape[0]
    contingency_table = pd.crosstab(data[col], data['Severity'])  # Create a contingency table
    chi2, p, dof, expected = chi2_contingency(contingency_table)  # Perform Chi-Square test

    n = contingency_table.values.sum()  # Total number of observations
    k = min(contingency_table.shape)  # Smaller of the number of rows or columns
    cramers_v = np.sqrt(chi2 / (n * (k - 1)))


    results_cat.append({
        'Features': col,
        'Chi-Square': chi2,
        'p-value': p,
        'Degrees of Freedom': dof,
        'cramers v': cramers_v,
        'Samples':x

    })

results_df = pd.DataFrame(results_cat)
results_df_sorted = results_df.sort_values(by='cramers v', ascending=False)

results_df_sorted.to_excel('cat_comp.xlsx',index=False)

print(results_df_sorted)



                            Features  Chi-Square       p-value  \
13                       Peritonitis   41.285256  1.083978e-09   
14                  Ketones_in_Urine   26.448921  7.680867e-06   
7                       Neutrophilia   23.754337  1.094485e-06   
3                             Nausea   23.170957  1.482190e-06   
4                   Loss_of_Appetite   20.563795  5.767666e-06   
11                       Free_Fluids   16.723254  4.324765e-05   
15                      RBC_in_Urine   10.653151  1.375718e-02   
9     Ipsilateral_Rebound_Tenderness    5.616847  1.778862e-02   
16                      WBC_in_Urine    4.896576  1.795289e-01   
6                              Stool    5.121086  7.726277e-02   
8               Lower_Right_Abd_Pain    2.658433  1.030021e-01   
12                        Psoas_Sign    1.582334  2.084248e-01   
10  Contralateral_Rebound_Tenderness    1.496745  2.211728e-01   
1                     Migratory_Pain    0.586144  4.439140e-01   
5         

In [38]:
results_cont=[]

df_cont=df[feat_cont]

# print(df_new_cont.columns)
for col in df_cont.columns:
    if col in target:
        continue
    data = df_cont[[col, 'Severity']].dropna()
    x=data.shape[0]
    group_0=data[data['Severity']==0][col]
    group_1=data[data['Severity']==1][col]
    stat, p_value = mannwhitneyu(group_0, group_1, alternative='two-sided')

    n1, n0 = len(group_1), len(group_0)
    r_rb = 1 - (2 * stat) / (n1 * n0)

    median_0 = np.median(group_0)
    median_1 = np.median(group_1)

    mean_0 = np.mean(group_0)
    mean_1 = np.mean(group_1)
    
    results_cont.append({'Feature': col, 'U_statistic': stat, 'p_value': p_value,'rank-biserial correlation': r_rb,'Mean (Positive Class)':mean_1,
                    'Mean(Negative Class)':mean_0,'Median(Positive Class)':median_1,'Median(Negative Class)':median_0,'Samples':x})

  

results_df = pd.DataFrame(results_cont)
results_df_sorted = results_df.sort_values(by='p_value', ascending=True)


results_df_sorted.to_excel('con_comp.xlsx',index=False)